# Vertiefung

Jetzt haben wir die allerersten Schritte getan und wagen uns an etwas komplexere Themen, bevor wir uns an die Tools für die KI herantasten. 

Zwei ganz wichtige Usecases für Anwendungen mit KI sind zum einen eine Chat-Funktion und das Bereitstellen von (eigenen) Daten. Da der Fokus bei diesem Workshop auf den Agenten liegt, wollen wir nur auf die einfachste Varianten einen Blick werfen. 

Zum Warm werden, schauen wir uns eine sehr simple Chat-Funktion an. Auch hierfür bietet LangChain eine Menge Hilfsmittel an. Grundsätzlich laufen Chats mit der KI so ab, dass man in jedem Request an die KI immer die gesamte Chat-Historie mitschickt. D.h. die KI (und die Software, die die KI bereitstellt) speichert nicht den Chatverlauf in der Regel. Das Speichern (im Arbeitsspeicher) der Historie übernimmt in LangChain-Jargon ein _Memory_. 


![LangChain Memory](https://python.langchain.com/v0.2/assets/images/message_history-4c13b8b9363beb4621d605bf6b5a34b4.png)

Hinweis: _Runnable_ ist ein Interface, das die Grundlage eines Gliedes einer Kette bildet. D.h. man kann Runnables verketten.

Beispielcode sagt mehr als tausend Worte.

In [ ]:
!pip install langchain langchain_openai langchain_community pypdf faiss-cpu python-dotenv

In [ ]:
from langchain_core.prompts import ChatPromptTemplate, MessagesPlaceholder
from langchain_core.output_parsers import StrOutputParser
import os
from langchain_openai import ChatOpenAI
from langchain_core.runnables.history import RunnableWithMessageHistory
from langchain.memory import ChatMessageHistory
from dotenv import load_dotenv

load_dotenv()
os.environ["OPENAI_API_KEY"] = os.getenv("OPENAI_API_KEY")

model = ChatOpenAI(model="gpt-4o")

parser = StrOutputParser()

system_template = "Du bist ein sehr hilfreicher Chatbot, der alles dafür tut, um dem Nutzer zu helfen."

prompt_template = ChatPromptTemplate.from_messages(
    [
        ("system", system_template), 
        MessagesPlaceholder(variable_name="chat_history"),
        ("user", "{input}")]
)

chat_history = ChatMessageHistory()

chain = prompt_template | model | parser

chain_with_message_history = RunnableWithMessageHistory(
    chain,
    lambda session_id: chat_history,
    input_messages_key="input",
    history_messages_key="chat_history",
)

while True:
    user_input = input("Deine Frage (gebe 'exit' zum Beenden ein): ")
    if user_input == "exit":
        break
    antwort = chain_with_message_history.invoke({"input": user_input, "chat_history": chat_history}
                                                , {"configurable": {"session_id": "unused"}})
    print(f"KI: {antwort}")